In [1]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
import requests
from github import Github
from git import Repo
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp
import configparser
from pprint import pprint
from datetime import datetime
#from gensim.models import Word2Vec

In [2]:
config = configparser.ConfigParser()
config.read('authentication.ini')
print(config.sections())
github_oauth = config['github']['api_key']

['github']


In [3]:
project_releases = pd.read_csv('volatile_projects_complete_links_limit10_filtered.csv')
project_releases.head()

,project_name,project_link,version_name,commit,timestamp
0,Dbeaver,https://github.com/dbeaver/dbeaver,21.1.4,"Commit(sha=""113a0a672f277a6e8181757a0c54f92d42...",29/7/2021 11:08
1,Dbeaver,https://github.com/dbeaver/dbeaver,21.1.3,"Commit(sha=""4430459a3fe06c6140aa40b71ddc41ddf8...",15/7/2021 8:06
2,Dbeaver,https://github.com/dbeaver/dbeaver,21.1.2,"Commit(sha=""b0693d44048a9c50e750b6df69cfe83fcb...",2/7/2021 13:34
3,Dbeaver,https://github.com/dbeaver/dbeaver,21.1.1,"Commit(sha=""073dfc26c7a065f5d5abf18be8cce8258a...",18/6/2021 13:50
4,Dbeaver,https://github.com/dbeaver/dbeaver,21.1.0,"Commit(sha=""17ce2d14317b1160ec9480da549028d182...",28/5/2021 5:16


In [238]:
project_refactoring_data = pd.read_csv('Effort_Estimation_Results_v2/Byte-buddy.csv')
project_refactoring_data.head()

,file,class,type,cbo,wmc,dit,rfc,lcom,totalMethods,staticMethods,...,anonymousClassesQty,subClassesQty,lambdasQty,uniqueWordsQty,modifiers,latest_commit,commit_compared_with,num_dependency,num_line_affected,actual_num_of_classes_touched
0,E:\SoftwareRemodularization\raw_sourcecode\Byt...,net.bytebuddy.utility.FileSystem,class,6,5,3,4,6,4,1,...,0,4,0,118,1025,ec813ad1a02b1b58154d5df9389ecaa786041d6c,eb4874b4bf26f6db61ed2040980ade9684ff06aa,54.0,238,3
1,E:\SoftwareRemodularization\raw_sourcecode\Byt...,net.bytebuddy.utility.Invoker,interface,0,2,1,0,1,2,0,...,0,0,0,87,1,52bfa03a9fdf85e1c051f41468872a8f2c29dcaf,db1773569e8b184b35a17c67aa647b204836680a,12.0,32,1
2,E:\SoftwareRemodularization\raw_sourcecode\Byt...,net.bytebuddy.utility.Invoker,interface,0,2,1,0,1,2,0,...,0,0,0,87,1,52bfa03a9fdf85e1c051f41468872a8f2c29dcaf,db1773569e8b184b35a17c67aa647b204836680a,12.0,32,2
3,E:\SoftwareRemodularization\raw_sourcecode\Byt...,net.bytebuddy.utility.Invoker,interface,0,2,1,0,1,2,0,...,0,0,0,87,1,ff69a57112d8ab06c18b1b2f87e8a9c714d80176,db1773569e8b184b35a17c67aa647b204836680a,12.0,32,1
4,E:\SoftwareRemodularization\raw_sourcecode\Byt...,net.bytebuddy.utility.Invoker,interface,0,2,1,0,1,2,0,...,0,0,0,87,1,ff69a57112d8ab06c18b1b2f87e8a9c714d80176,db1773569e8b184b35a17c67aa647b204836680a,12.0,32,2


In [239]:
unique_releases_commit = pd.unique(project_refactoring_data['latest_commit'])
unique_commit = pd.unique(project_refactoring_data['commit_compared_with'])

In [240]:
print(unique_commit)
print(unique_releases_commit)

['eb4874b4bf26f6db61ed2040980ade9684ff06aa'
 'db1773569e8b184b35a17c67aa647b204836680a'
 '968b13db5a63f18fb04eb98f875486a312a6b3c3'
 'd66264cfac45c7e1657ed0dec0df4d851f7bce8c']
['ec813ad1a02b1b58154d5df9389ecaa786041d6c'
 '52bfa03a9fdf85e1c051f41468872a8f2c29dcaf'
 'ff69a57112d8ab06c18b1b2f87e8a9c714d80176'
 '75e2bb83f5f797d5cedf3fbd18a3aa8e1dff1061']


In [241]:
unique_commits = list(set(unique_commit).symmetric_difference(set(unique_releases_commit)))

In [242]:
unique_commits

['75e2bb83f5f797d5cedf3fbd18a3aa8e1dff1061',
 'eb4874b4bf26f6db61ed2040980ade9684ff06aa',
 '52bfa03a9fdf85e1c051f41468872a8f2c29dcaf',
 'ec813ad1a02b1b58154d5df9389ecaa786041d6c',
 'ff69a57112d8ab06c18b1b2f87e8a9c714d80176',
 'd66264cfac45c7e1657ed0dec0df4d851f7bce8c',
 '968b13db5a63f18fb04eb98f875486a312a6b3c3',
 'db1773569e8b184b35a17c67aa647b204836680a']

In [244]:
project_link = "https://github.com/raphw/byte-buddy"

In [245]:
project_owner = 'raphw'
project_name_short = 'byte-buddy'

## To obtain total number of pages of commits

In [246]:
query_url = f"https://api.github.com/repos/{project_owner}/{project_name_short}/commits?per_page=100"

headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers)
print(len(r.json()))

counter = 0
project_name = project_name_short

try:
    os.mkdir(f'raw_commits_github_json/{project_name}/')
except:
    pass

with open(f'raw_commits_github_json/{project_name}/{project_name}_{counter}.json', 'w', encoding='utf-8') as f:
    json.dump(r.json(), f, ensure_ascii=False, indent=4)
    
while 'next' in r.links:
    query_url = r.links['next']['url']
    r = requests.get(query_url, headers=headers)
    counter += 1
    with open(f'raw_commits_github_json/{project_name}/{project_name}_{counter}.json', 'a', encoding='utf-8') as f:
        json.dump(r.json(), f, ensure_ascii=False, indent=4)
    
    
f.close()

100


In [247]:
main_arr = []

root_dir = f'raw_commits_github_json/{project_name_short}'
data_counter = 0
for filename in os.listdir(root_dir):
    print(filename)

    with open(f'{root_dir}/{filename}', encoding='utf-8') as f:
        data = json.load(f)
        #print(len(data))
        data_counter += len(data)
        
        
        for commits in data:
            try:
                temp_arr = []
                temp_arr.append(commits['sha'])
                temp_arr.append(commits['commit']['author']['name'])
                temp_arr.append(commits['commit']['author']['email'])
                temp_arr.append(commits['commit']['author']['date'])
                temp_arr.append(commits['author']['login'])
                temp_arr.append(commits['commit']['message'])
                temp_arr.append(commits['parents'][0]['sha'])

                main_arr.append(temp_arr)
            except:
                pass
            

byte-buddy_0.json
byte-buddy_1.json
byte-buddy_10.json
byte-buddy_11.json
byte-buddy_12.json
byte-buddy_13.json
byte-buddy_14.json
byte-buddy_15.json
byte-buddy_16.json
byte-buddy_17.json
byte-buddy_18.json
byte-buddy_19.json
byte-buddy_2.json
byte-buddy_20.json
byte-buddy_21.json
byte-buddy_22.json
byte-buddy_23.json
byte-buddy_24.json
byte-buddy_25.json
byte-buddy_26.json
byte-buddy_27.json
byte-buddy_28.json
byte-buddy_29.json
byte-buddy_3.json
byte-buddy_30.json
byte-buddy_31.json
byte-buddy_32.json
byte-buddy_33.json
byte-buddy_34.json
byte-buddy_35.json
byte-buddy_36.json
byte-buddy_37.json
byte-buddy_38.json
byte-buddy_39.json
byte-buddy_4.json
byte-buddy_40.json
byte-buddy_41.json
byte-buddy_42.json
byte-buddy_43.json
byte-buddy_44.json
byte-buddy_45.json
byte-buddy_46.json
byte-buddy_47.json
byte-buddy_48.json
byte-buddy_49.json
byte-buddy_5.json
byte-buddy_50.json
byte-buddy_51.json
byte-buddy_52.json
byte-buddy_53.json
byte-buddy_54.json
byte-buddy_55.json
byte-buddy_56.json

In [248]:
data_counter

5993

In [249]:
data[1]['parents'][0]['sha']

'e98b5011ddaf0d34fda1c33b1268acff12423318'

In [250]:
okhttp_commits_df = pd.DataFrame(main_arr)

In [251]:
okhttp_commits_df.columns = ['sha','name','email','date','login','message', 'parent_sha']

## Creating the dictionary for reference Parent SHA -> Parent SHA Date

In [252]:
parent_sha_dict = {}
for row in okhttp_commits_df.iterrows():
    try:
        # if the commit has a child
        parent_sha_date = okhttp_commits_df[okhttp_commits_df['sha'] == row[1]['parent_sha']]['date'].values[0]
        parent_sha_dict[row[1]['sha']] = parent_sha_date
    except:
        # if the commit does not have a child
        parent_sha_dict[row[1]['sha']] = None
        

In [253]:
parent_sha_dict

{'7402e597870dfacce6fe13a1390070c38bbb2317': '2021-11-09T20:51:37Z',
 'c1de36544196314fd43f54f97f5bf3c309460c7e': '2021-11-09T20:51:26Z',
 '4cdbfc0f266b510c00872ebca8b9620b496f59e1': '2021-11-09T20:44:16Z',
 'c004000c80e2184ad04cc7b6e4332d0f4007f503': '2021-11-09T20:44:14Z',
 'ef26cf09ddc6a9f0b800b03c0c45de9474416f0f': '2021-11-09T20:24:43Z',
 '952e04531cce73c479be886b8a7f6c5ea8b60ed5': '2021-11-09T07:47:15Z',
 'b6d899c407c4211b8dee45f69fef62281756beb6': '2021-11-07T20:18:12Z',
 '3ea315db38349d9b24daa0aa42c4d06baf23d35c': '2021-11-07T20:15:11Z',
 '9bac44eb4b140fe5c967d74312672e1b35b4f1da': '2021-11-07T20:14:52Z',
 '8c7c3ce45a50170d5b229c709c402ce42fa5f262': '2021-11-07T20:13:18Z',
 'd10a5b7b031c1f3bb7dc828e3ae69ff8b544d0b6': '2021-11-07T20:12:14Z',
 'a9603b49d746d87ad11654754598ff2293772657': '2021-11-07T20:01:18Z',
 '54b22a380cfe10b1849bc4a87c38e23fd4258fc9': '2021-11-05T21:18:33Z',
 'd865202d6d9a5b240c60aa9a99888df8946eb1a1': '2021-11-05T21:06:13Z',
 'fbc8f6ea6ce59884042c5148a95f67a4

In [254]:
def label_parent_date(row):
    return parent_sha_dict[row['sha']]

okhttp_commits_df['parent_date'] = okhttp_commits_df.apply(lambda row: label_parent_date(row), axis=1)

In [255]:
print(len(okhttp_commits_df))

4325


In [256]:
okhttp_commits_df.dtypes

sha            object
name           object
email          object
date           object
login          object
message        object
parent_sha     object
parent_date    object
dtype: object

In [257]:
okhttp_commits_df['date'] = pd.to_datetime(okhttp_commits_df['date'], infer_datetime_format=True)
okhttp_commits_df['parent_date'] = pd.to_datetime(okhttp_commits_df['parent_date'], infer_datetime_format=True)

In [258]:
okhttp_commits_df['time_taken'] = (okhttp_commits_df['date'] - okhttp_commits_df['parent_date']).astype('timedelta64[h]')

In [259]:
okhttp_commits_df.head()

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken
0,7402e597870dfacce6fe13a1390070c38bbb2317,Rafael Winterhalter,rafael.wth@gmail.com,2021-11-10 21:13:31,raphw,Better error message if class file version is ...,c1de36544196314fd43f54f97f5bf3c309460c7e,2021-11-09 20:51:37,24.0
1,c1de36544196314fd43f54f97f5bf3c309460c7e,Rafael Winterhalter,rafael.wth@gmail.com,2021-11-09 20:51:37,raphw,Fix typo.,4cdbfc0f266b510c00872ebca8b9620b496f59e1,2021-11-09 20:51:26,0.0
2,4cdbfc0f266b510c00872ebca8b9620b496f59e1,Rafael Winterhalter,rafael.wth@gmail.com,2021-11-09 20:51:26,raphw,Update release notes and internal Byte Buddy.,c004000c80e2184ad04cc7b6e4332d0f4007f503,2021-11-09 20:44:16,0.0
3,c004000c80e2184ad04cc7b6e4332d0f4007f503,Rafael Winterhalter,rafael.wth@gmail.com,2021-11-09 20:44:16,raphw,[maven-release-plugin] prepare for next develo...,ef26cf09ddc6a9f0b800b03c0c45de9474416f0f,2021-11-09 20:44:14,0.0
4,ef26cf09ddc6a9f0b800b03c0c45de9474416f0f,Rafael Winterhalter,rafael.wth@gmail.com,2021-11-09 20:44:14,raphw,[maven-release-plugin] prepare release byte-bu...,952e04531cce73c479be886b8a7f6c5ea8b60ed5,2021-11-09 20:24:43,0.0


In [260]:
def check_if_refactoring_present_in_commit(row):
    if row['sha'] in unique_commits:
        return 1
    return 0

In [261]:
okhttp_commits_df['contains_refactoring'] = okhttp_commits_df.apply(lambda row: check_if_refactoring_present_in_commit(row), axis=1)

In [262]:
okhttp_commits_df.head()

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken,contains_refactoring
0,7402e597870dfacce6fe13a1390070c38bbb2317,Rafael Winterhalter,rafael.wth@gmail.com,2021-11-10 21:13:31,raphw,Better error message if class file version is ...,c1de36544196314fd43f54f97f5bf3c309460c7e,2021-11-09 20:51:37,24.0,0
1,c1de36544196314fd43f54f97f5bf3c309460c7e,Rafael Winterhalter,rafael.wth@gmail.com,2021-11-09 20:51:37,raphw,Fix typo.,4cdbfc0f266b510c00872ebca8b9620b496f59e1,2021-11-09 20:51:26,0.0,0
2,4cdbfc0f266b510c00872ebca8b9620b496f59e1,Rafael Winterhalter,rafael.wth@gmail.com,2021-11-09 20:51:26,raphw,Update release notes and internal Byte Buddy.,c004000c80e2184ad04cc7b6e4332d0f4007f503,2021-11-09 20:44:16,0.0,0
3,c004000c80e2184ad04cc7b6e4332d0f4007f503,Rafael Winterhalter,rafael.wth@gmail.com,2021-11-09 20:44:16,raphw,[maven-release-plugin] prepare for next develo...,ef26cf09ddc6a9f0b800b03c0c45de9474416f0f,2021-11-09 20:44:14,0.0,0
4,ef26cf09ddc6a9f0b800b03c0c45de9474416f0f,Rafael Winterhalter,rafael.wth@gmail.com,2021-11-09 20:44:14,raphw,[maven-release-plugin] prepare release byte-bu...,952e04531cce73c479be886b8a7f6c5ea8b60ed5,2021-11-09 20:24:43,0.0,0


In [263]:
okhttp_commits_df['contains_refactoring'].value_counts()

0    4317
1       8
Name: contains_refactoring, dtype: int64

In [264]:
okhttp_commits_df.to_csv(f'raw_commits_github_json/{project_name_short}/{project_name_short}_commits.csv',index=False)

In [265]:
okhttp_commits_df = pd.read_csv(f'raw_commits_github_json/{project_name_short}/{project_name_short}_commits.csv')

In [266]:
okhttp_commits_df.head()

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken,contains_refactoring
0,7402e597870dfacce6fe13a1390070c38bbb2317,Rafael Winterhalter,rafael.wth@gmail.com,2021-11-10 21:13:31,raphw,Better error message if class file version is ...,c1de36544196314fd43f54f97f5bf3c309460c7e,2021-11-09 20:51:37,24.0,0
1,c1de36544196314fd43f54f97f5bf3c309460c7e,Rafael Winterhalter,rafael.wth@gmail.com,2021-11-09 20:51:37,raphw,Fix typo.,4cdbfc0f266b510c00872ebca8b9620b496f59e1,2021-11-09 20:51:26,0.0,0
2,4cdbfc0f266b510c00872ebca8b9620b496f59e1,Rafael Winterhalter,rafael.wth@gmail.com,2021-11-09 20:51:26,raphw,Update release notes and internal Byte Buddy.,c004000c80e2184ad04cc7b6e4332d0f4007f503,2021-11-09 20:44:16,0.0,0
3,c004000c80e2184ad04cc7b6e4332d0f4007f503,Rafael Winterhalter,rafael.wth@gmail.com,2021-11-09 20:44:16,raphw,[maven-release-plugin] prepare for next develo...,ef26cf09ddc6a9f0b800b03c0c45de9474416f0f,2021-11-09 20:44:14,0.0,0
4,ef26cf09ddc6a9f0b800b03c0c45de9474416f0f,Rafael Winterhalter,rafael.wth@gmail.com,2021-11-09 20:44:14,raphw,[maven-release-plugin] prepare release byte-bu...,952e04531cce73c479be886b8a7f6c5ea8b60ed5,2021-11-09 20:24:43,0.0,0


In [267]:
query_url = f"https://api.github.com/rate_limit"
print(query_url)
#query_url = f"https://api.github.com/repos/{repo}"
headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers)
pprint(r.json())
print(len(r.json()))

https://api.github.com/rate_limit
{'rate': {'limit': 5000, 'remaining': 4430, 'reset': 1636935300, 'used': 570},
 'resources': {'actions_runner_registration': {'limit': 10000,
                                               'remaining': 10000,
                                               'reset': 1636936443,
                                               'used': 0},
               'code_scanning_upload': {'limit': 500,
                                        'remaining': 500,
                                        'reset': 1636936443,
                                        'used': 0},
               'core': {'limit': 5000,
                        'remaining': 4430,
                        'reset': 1636935300,
                        'used': 570},
               'graphql': {'limit': 5000,
                           'remaining': 5000,
                           'reset': 1636936443,
                           'used': 0},
               'integration_manifest': {'limit': 5000,
          